In [ ]:
from PIL import Image, ImageOps, ImageFilter
import cv2
import numpy as np
import os

In [ ]:
PATH_WATERMARK = 'watermark.png'
PATH_FOLDER_RAW = 'images_raw'
PATH_FOLDER_EDITED = 'images_edited'

# Functions

In [ ]:
##############################################################################################################################

def resize_img(img, target, by=''):

    w, h = img.size

    if by=='height':
        scale = target / h
        target_w, target_h = int(w * scale), target
    elif by=='width':
        scale = target / w
        target_w, target_h = target, int(h * scale)
    
    img_resized = img.resize((target_w, target_h), Image.ANTIALIAS)

    return img_resized

##############################################################################################################################

def crop_img_square(img, length_sides, by=''):
    
    w, h = img.size

    # left, top, right, bottom
    if by=='horizontal':
        crop_size = int((w - length_sides)/2)
        border = (crop_size, 0, crop_size, 0)

    elif by=='vertical':
        crop_size = int((h - length_sides)/2)
        border = (0, crop_size, 0, crop_size)
    
    img_square = ImageOps.crop(img, border)

    return img_square

##############################################################################################################################

def combine_images(img_back, img_front):

    w_back, h_back = img_back.size
    w_front, h_front = img_front.size

    img_combined = img_back.filter(ImageFilter.GaussianBlur(15))
    img_combined.paste(img_front, (int((w_back - w_front)/2), int((h_back - h_front)/2)))
    
    return img_combined

##############################################################################################################################

def watermark_image(img_combined, img_watermark, scale_wm=None):

    if scale_wm!=None:

        # re-scale watermark
        wm_width, wm_height = int(img_watermark.shape[1] * scale_wm), int(img_watermark.shape[0] * scale_wm)
        wm_dim = (wm_width, wm_height)
        img_watermark = cv2.resize(img_watermark, wm_dim, interpolation=cv2.INTER_AREA)

    h_img, w_img, _ = img_combined.shape
    center_y, center_x = int(h_img/2), int(w_img/2)

    h_wm, w_wm, _ = img_watermark.shape
    top_y, left_x = center_y - int(h_wm/2), center_x - int(w_wm/2)
    bottom_y, right_x = top_y + h_wm, left_x + w_wm

    roi = img_combined[top_y:bottom_y, left_x:right_x]
    result = cv2.addWeighted(roi, 1, img_watermark, 0.3, 1)
    img_combined[top_y:bottom_y, left_x:right_x] = result

    return img_combined
    
##############################################################################################################################

# Driver Code

In [ ]:
# find all folders in raw directory
folders_raw = [f for f in os.listdir(PATH_FOLDER_RAW)]

# filter out '.DS_Store' files (for Mac users)
folders_raw = [f for f in folders_raw if f!='.DS_Store']

print(folders_raw)

### Square Crop, & Watermark Images

In [ ]:
for folder in folders_raw:

    # raw folder directory
    path_folder_raw = f'{PATH_FOLDER_RAW}/{folder}'

    # create folder to store edited images
    path_folder_edited = f'{PATH_FOLDER_EDITED}/{folder}'
    if not os.path.exists(path_folder_edited):
        os.makedirs(path_folder_edited)

    # find all images in raw folder
    images = [img for img in os.listdir(path_folder_raw)]
    # filter to only image type files
    images = [img for img in images if (('.jpeg' in img) or ('.jpg' in img) or ('.png' in img))]

    for img in images:

        # read raw image
        path_image_raw = f'{path_folder_raw}/{img}'
        img_raw = Image.open(path_image_raw)
        w, h = img_raw.size

        # generate & resize front & background images (from raw image)
        # crop background image to square
        if w > h:
            img_resized = resize_img(img_raw, 1080, by='height')
            img_resized = crop_img_square(img_resized, 1080, by='horizontal')
        else:
            img_resized = resize_img(img_raw, 1080, by='width')
            img_resized = crop_img_square(img_resized, 1080, by='vertical')

        # convert combined image to cv2 format
        img_resized = cv2.cvtColor(np.array(img_resized), cv2.COLOR_RGB2BGR)

        # read watermark image as cv2 format
        img_watermark = cv2.imread(PATH_WATERMARK)

        # add watermark to image
        img_combined = watermark_image(img_resized, img_watermark, scale_wm=0.25)

        # save edited image
        path_image_edited = f'{path_folder_edited}/edited-{img}'
        cv2.imwrite(path_image_edited, img_combined)

### Square Crop, Blurred Border, & Watermark Images

In [ ]:
# for folder in folders_raw:

#     # raw folder directory
#     path_folder_raw = f'{PATH_FOLDER_RAW}/{folder}'

#     # create folder to store edited images
#     path_folder_edited = f'{PATH_FOLDER_EDITED}/{folder}'
#     if not os.path.exists(path_folder_edited):
#         os.makedirs(path_folder_edited)

#     # find all images in raw folder
#     images = [img for img in os.listdir(path_folder_raw)]
#     # filter to only image type files
#     images = [img for img in images if (('.jpeg' in img) or ('.jpg' in img) or ('.png' in img))]

#     for img in images:

#         # read raw image
#         path_image_raw = f'{path_folder_raw}/{img}'
#         img_raw = Image.open(path_image_raw)
#         w, h = img_raw.size

#         # generate & resize front & background images (from raw image)
#         # crop background image to square
#         if w > h:
#             img_back = resize_img(img_raw, 1080, by='height')
#             img_front = resize_img(img_raw, 1080, by='width')
#             img_back = crop_img_square(img_back, 1080, by='horizontal')
#         else:
#             img_back = resize_img(img_raw, 1080, by='width')
#             img_front = resize_img(img_raw, 1080, by='height')
#             img_back = crop_img_square(img_back, 1080, by='vertical')

#         # combine background & front images
#         img_combined = combine_images(img_back, img_front)

#         # convert combined image to cv2 format
#         img_combined = cv2.cvtColor(np.array(img_combined), cv2.COLOR_RGB2BGR)

#         # read watermark image as cv2 format
#         img_watermark = cv2.imread(PATH_WATERMARK)

#         # add watermark to image
#         img_combined = watermark_image(img_combined, img_watermark, scale_wm=0.25)

#         # save edited image
#         path_image_edited = f'{path_folder_edited}/edited-{img}'
#         cv2.imwrite(path_image_edited, img_combined)